In [2]:
import numpy as np

In [51]:
amino_mass = {'G': 57.,'A': 71., 'S':87., 
              'P':97.1, 'V':99.1, 
              'T': 101.1, 'C':103., 'I': 113.1, 
              'L': 113.0, 'N': 114., 
              'D': 115., 'Q': 128.1, 'K': 128.1, 'E': 129., 
              'M': 131., 'H': 137.1, 'F': 147.1, 'R': 156.1,
              'Y': 163.1, 'W': 186.1}
amino_lst = list(amino_mass.keys())

def cyclo_spectrum(peptide):
    global amino_lst
    global amino_mass
    pref_mass = [0]
    for i in range(len(peptide)):
        for j in range(20):
            if amino_lst[j] == peptide[i]:
                #print(pref_mass[i-1]+amino_mass[amino_lst[j]], i, j)
                pref_mass.append(pref_mass[i]+amino_mass[amino_lst[j]])
    #print(pref_mass)        
    peptide_mass = max(pref_mass)
    cycle_spec = [0]
    for i in range(0, len(peptide)):
        for j in range(i+1, len(peptide)+1):
            res = pref_mass[j]-pref_mass[i]
            cycle_spec.append(round(res,1))
            if i > 0 and j < len(peptide):
                cycle_spec.append(round((peptide_mass - res),1))
    return sorted(cycle_spec)

cyclo_spectrum('NQEL')

[0,
 113.0,
 114.0,
 128.0,
 129.0,
 227.0,
 242.0,
 242.0,
 257.0,
 355.0,
 356.0,
 370.0,
 371.0,
 484.0]

In [52]:
def mass(peptide):
    res = 0
    for aa in peptide:
        res += amino_mass[aa]
    return res

def cyclopeptide_seq(spectre):
    peptides = []
    saved = []
    if not saved:
        peptides.extend(amino_lst)
    while len(peptides) > 0:
        for peptide in peptides:
            if mass(peptide) in spectre:
                if cyclo_spectrum(peptide) == spectre:
                    return peptide
                saved.append(peptide) 
                peptides.remove(peptide)
            else:
                peptides.remove(peptide)
        for saved_peptide in saved:
            for aa in amino_lst:
                peptides.append(saved_peptide + aa)

spectre = cyclo_spectrum('NQEL')
cyclopeptide_seq(spectre)

'NQEL'

In [102]:
def check_in_with_delta(obj, lst):
    delta = 0.1
    check_lst = np.linspace(obj-delta, obj+delta, 3)
    for ch in check_lst:
        if ch in lst:
            return True
    else:
        return False
    
lst = [1.1, 2]
check_in_with_delta(1., lst)

True

In [103]:
def find_score(peptide, spectre):
    spec_theor = cyclo_spectrum(peptide)
    score = 0
    for x in spec_theor:
        if check_in_with_delta(x, spectre):
            score += 1
    return score

In [117]:
import itertools as it

def trim(peptides, spectre):
    score_lst = []
    for peptide in peptides:
        score_lst.append(find_score(peptide, spectre))
    res = [x for _, x in sorted(zip(score_lst, peptides))]
    #print(res)
    return res[len(peptides)-200 : len(peptides)] 

def real_cyclopep_seq(spectre):
    peptides = []
    score_max = float('-inf')
    flag = True
    ans = ''
    peptides.extend(map(''.join, list(it.product(amino_lst, repeat=3))))
    while len(peptides) > 0:
        res = trim(peptides, spectre)
        for peptide in res:
            score = find_score(peptide, spectre)
            if score > score_max:
                score_max = score
                ans = peptide
            if mass(peptide) - 300 > max(spectre):
                flag = False
        if mass(ans) == max(spectre):
            return ans
        if flag:
            peptides = []
            for pep in res:
                for aa in amino_lst:
                    peptides.append(pep + aa)
        else:
            return ans

In [118]:
data = []
with open('data/Spectrum.txt', 'r') as f:
    for line in f:
        data.extend(list(map(float, (line.rstrip()).split())))
spectre = data

In [119]:
real_cyclopep_seq(spectre)

'YYTQYLILYQT'

In [121]:
find_score('YWWTFNKNGA', spectre)

9